# Setup

In [1]:
!pip install git+https://github.com/hafidhrendyanto/gpt2-absa.git

  Cloning https://github.com/hafidhrendyanto/gpt2-absa.git to /tmp/pip-req-build-duqzyf6v
  Running command git clone -q https://github.com/hafidhrendyanto/gpt2-absa.git /tmp/pip-req-build-duqzyf6v
     |████████████████████████████████| 3.1 MB 7.1 MB/s 
     |████████████████████████████████| 596 kB 45.6 MB/s 
     |████████████████████████████████| 3.3 MB 60.4 MB/s 
     |████████████████████████████████| 59 kB 8.3 MB/s 
     |████████████████████████████████| 895 kB 60.0 MB/s 
  Created wheel for gpt2absa: filename=gpt2absa-0.1.0-py3-none-any.whl size=4464 sha256=f11981f3b9a2b55e721dff21b75dfa09e52e1975437e961dfacf5a809f2bb7c1
  Stored in directory: /tmp/pip-ephem-wheel-cache-tpl716z5/wheels/59/3f/21/821ef35f74032bad30c93d305f29f28fff780907ca4176f4ed
Successfully built gpt2absa
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# Code Sample

In [3]:
from gpt2absa.constant import restaurant_aspect_categories, laptop_aspect_categories
from gpt2absa import aspect_polarity_pair

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [4]:
from transformers import TFAutoModelWithLMHead
model = TFAutoModelWithLMHead.from_pretrained("hafidhrendyanto/gpt2-absa")

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_tf_auto.py:429: FutureWarning: The class `TFAutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `TFAutoModelForCausalLM` for causal language models, `TFAutoModelForMaskedLM` for masked language models and `TFAutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Downloading:   0%|          | 0.00/938 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/475M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at hafidhrendyanto/gpt2-absa.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


## Restaurant domain

Aspect Based Sentiment Analysis (ABSA) can be devided into many sub-task. Two of which are *sentiment polarity classification* and *aspect category detection*. The samples for restaurant and laptop dommain solve both sub-task while the sample for hotel domain only solve the former (*sentiment polarity classification*).

In [5]:
aspect_polarity_pair(model, "Excellent food, although the interior could use some help.", restaurant_aspect_categories)

{'food#quality': 'positive'}

In [6]:
aspect_polarity_pair(model, "Price is high but the food is good, so I would come back again.", restaurant_aspect_categories)

{'food#quality': 'positive', 'restaurant#general': 'positive'}

In [7]:
aspect_polarity_pair(model, "You can order your drinks however you like, and the bartender is very helpfull", restaurant_aspect_categories)

{'drinks#quality': 'positive', 'service#general': 'positive'}

In [8]:
aspect_polarity_pair(model, "I come here at noon", restaurant_aspect_categories)

{'no#sentiment': 'neutral', 'restaurant#general': 'neutral'}

## Laptop domain

You can influence the amount of the selected aspect by changing the `treshold` argument of the function. Because there is more aspect candidate in the laptop domain, the optimal value for this argument is `0.35`

In [9]:
aspect_polarity_pair(model, "Looks good, well built, and very good speed.", laptop_aspect_categories)

{'laptop#design_features': 'positive',
 'laptop#general': 'positive',
 'laptop#operation_performance': 'positive',
 'laptop#quality': 'positive',
 'laptop#usability': 'positive'}

In [10]:
aspect_polarity_pair(model, "Looks good, well built, and very good speed.", laptop_aspect_categories, treshold=0.35)

{'laptop#design_features': 'positive',
 'laptop#general': 'positive',
 'laptop#operation_performance': 'positive',
 'laptop#quality': 'positive'}

In [11]:
aspect_polarity_pair(model, "This is the first laptop I've owned, althougth I used several at my previous job.", laptop_aspect_categories, treshold=0.35)

{'no#sentiment': 'neutral'}

In [12]:
aspect_polarity_pair(model, "Save your money and get an android: works better, more support", laptop_aspect_categories, treshold=0.35)

{'laptop#general': 'negative'}

In [13]:
aspect_polarity_pair(model, "Very fast and reliable, but the design is rather weird", laptop_aspect_categories, treshold=0.35)

{'laptop#design_features': 'negative',
 'laptop#general': 'positive',
 'laptop#operation_performance': 'positive'}

## Hotel domain

The model is only fine-tuned using SemEval 2016 Task 5 Restaurant and Laptop domain. The samples for hotel domain below shows the solutions cross domain generalization capabilities **for sentiment classification task**. You can use this solution on many other domain by defining the aspect categories that you want to evaluate on that domain.

Its must be noted that this domain generalization capability only applies to *sentiment polarity classification*. Currently, the *aspect category detection* module for this solution don't have that capability/features.

In [20]:
from gpt2absa import sentiment_polarity_classification

In [19]:
aspect_categories = ['hotel#room', 'hotel#view', 'hotel#staff', 'hotel#price', 'hotel#general']

In [21]:
text = "The room in this hotel is very beautiful, the view is impeccable but the price is so high and the staff can be grumpy sometimes."
sentiment_polarity_classification(model, text, aspect_categories)

{'hotel#general': 'positive',
 'hotel#price': 'negative',
 'hotel#room': 'positive',
 'hotel#staff': 'negative',
 'hotel#view': 'positive'}

In [22]:
text = "This is the best hotel I've been, all around great!"
sentiment_polarity_classification(model, text, aspect_categories)

{'hotel#general': 'positive',
 'hotel#price': 'positive',
 'hotel#room': 'positive',
 'hotel#staff': 'positive',
 'hotel#view': 'positive'}